In [9]:
import cv2
import numpy as np
#import os

# Define const.
ESC_KEY = 27     # Esc
INTERVAL= 33     # delay

WINDOW_ORG = "org"
WINDOW_BACK = "back"
WINDOW_DIFF = "diff"
WINDOW_CROPPED = "crpd"
image_dir='./image_dir/'
image_file='img_%s.png'
FILE_ORG = "macaque1.mp4" #Video file name

#Prepare the windows
cv2.namedWindow(WINDOW_ORG)
cv2.namedWindow(WINDOW_DIFF)
cv2.namedWindow(WINDOW_CROPPED)
fgbg = cv2.bgsegm.createBackgroundSubtractorGMG()
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
# Read the org video
mov_org = cv2.VideoCapture(FILE_ORG)

# Read the first frame
has_next, i_frame = mov_org.read()

i = 0
# Looping of BackgroundSubtractor 
while has_next == True:
   
    diff_frame = fgbg.apply(i_frame)
    diff_frame = cv2.morphologyEx(diff_frame, cv2.MORPH_OPEN, kernel)
   
    thresh = cv2.threshold(diff_frame, 10, 255, cv2.THRESH_BINARY)[1]
    contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[1]
    
    j = 0
    for pt in contours:
        x,y,w,h = cv2.boundingRect(pt)
        if w < 100: continue
        if x < 20 or y < 20:
            #cv2.rectangle(i_frame, (x, y), (x+w, y+h),(0, 255, 0), 3)
            crpd_i_frame = i_frame[y:y+h,x:x+w]
            cv2.imshow(WINDOW_CROPPED, crpd_i_frame)
            cv2.imwrite(image_dir+str(j)+image_file % str(i).zfill(6), crpd_i_frame)
            j+=1
        else:
            crpd_i_frame = i_frame[y-20:y+h+20,x-20:x+w+20]
            cv2.imshow(WINDOW_CROPPED, crpd_i_frame)
            cv2.imwrite(image_dir+str(j)+image_file % str(i).zfill(6), crpd_i_frame)
            j+=1
                    
        
    cv2.imshow(WINDOW_ORG, i_frame)
   
    cv2.imshow(WINDOW_DIFF, diff_frame.astype(np.uint8))
   
    i+=1
    # Finish when Esc key interrupts
    key = cv2.waitKey(INTERVAL)
    if key == ESC_KEY:
        break

    # Read the next frame
    has_next, i_frame = mov_org.read()

# Ending process
cv2.destroyAllWindows()
mov_org.release()